<a href="https://colab.research.google.com/github/TMMichaelsen/AI_Chatbot/blob/main/Chatbot_OGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot de Atendimento - Ouvidoria-Geral do Estado de Minas Gerais

## Instalação do SDK Google

In [1]:
# Instalação do pacote de IA Generativa
!pip install -U -q google-generativeai

# Importação das bibliotecas utilizadas
import google.generativeai as genai
from google.colab import userdata
import pandas as pd
import numpy as np

# Configuração da API
GOOGLE_API_KEY = userdata.get('API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Configurações do Modelo Generativo

In [2]:
#Listando os modelos generativos disponíveis

print('Modelos Generative:\n')
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

Modelos Generative:

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [8]:
# Definição do modelo

gen_model_name = 'gemini-1.5-pro-latest'

# Configurações de geração

generation_config = {
  "candidate_count": 1,
  "temperature": 0.6
}

# Configurações de segurança

safety_settings = {
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE'
}

# Instruções de sistema

system_instruction = """
Você é uma assistente virtual para a Ouvidoria-Geral do Estado de Minas Gerais - OGE chamada Bel.
Você deve buscar suas informações através do site https://www.ouvidoriageral.mg.gov.br/, bem como demais legislações pertinentes ao Governo do Estado de Minas Gerais.
Você deve ser sempre cordial, e utilizar linguagem simples em sua resposta.
Você recebe em seu prompt uma manifestação ou uma dúvida de um cidadão.
Você deve avaliar se a tratativa desta manifestação é de competência do Governo do Estado de Minas Gerais, ou competência diversa, indicando o canal correto para manifestar.
Caso seja competência do Estado de Minas Gerais, você deve indicar qual a Ouvidoria Temática da OGE que deve tratar esta manifestação.
As Ouvidorias Temáticas disponíveis são:
* Ouvidoria Ambiental e Agropecuária;
* Ouvidoria Educacional;
* Ouvidoria de Fazenda, Licitações e Patrimônio Público;
* Ouvidoria do Sistema Penitenciário e Socioeducativo;
* Ouvidoria de Polícia, subdividida entre as seguintes Assessorias: Polícia Militar de Minas Gerais, Polícia Civil de Minas Gerais, Corpo de Bombeiros Militares de Minas Gerais;
* Ouvidoria de Saúde;
* Ouvidoria de Prevenção e Combate ao Assédio Moral e Sexual;
* Ouvidoria de Desenvolvimento Econômico, Infraestrutura e Desenvolvimento Social;
* Ouvidoria de Prevenção e Combate à Corrupção.
As tipologias de manifestação são:
* Denúncia;
* Reclamação;
* Solicitação;
* Sugestão;
* Elogio;
* Simplifique
"""

In [9]:
# Instanciando o modelo generativo conforme configurações prévias

gen_model = genai.GenerativeModel(model_name = gen_model_name,
                                  generation_config = generation_config,
                                  safety_settings = safety_settings,
                                  system_instruction = system_instruction)

## Configurações do Modelo Embedded

In [5]:
#Listando os modelos embedding disponíveis

print('Modelos Embedding:\n')
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

Modelos Embedding:

models/embedding-001
models/text-embedding-004


Lembre-se de importar o arquivo "Perguntas Frequentes.xlsx" ao Google Colab antes de executar as etapas a seguir.

In [7]:
# Definição do modelo

emb_model_name = 'models/text-embedding-004'

confianca_considerada = 0.75

# Importando DataFrame de Perguntas Frequentes

df = pd.read_excel('Perguntas Frequentes.xlsx')   # Arquivo disponibilizado no repositório. Adicione ao Colab para execução.

# Calculando Embeddings das questões e respostas

def embed_fn(title, text):
  return genai.embed_content(model=emb_model_name,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

df["Embeddings"] = df.apply(lambda row: embed_fn(row["PERGUNTA"], row["RESPOSTA"]), axis=1)

In [10]:
def emb_model(prompt, base, model):
  embeddings = genai.embed_content(model=model,
                                   content=prompt,
                                   task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embeddings)
  confianca = max(produtos_escalares)
  indice = np.argmax(produtos_escalares)

  return confianca, indice

## Inicialização do Chatbot

In [12]:
gen_chat = gen_model.start_chat(history=[])

prompt = input('Digite sua pergunta, ou escreva "FIM" para finalizar: ')

while prompt.lower() != "fim":
  confianca, indice = emb_model(prompt, df, emb_model_name)
  if confianca >= confianca_considerada:
    response = df.iloc[indice]["RESPOSTA"]
  else:
    response = gen_chat.send_message(prompt).text

  print(f'\nResposta: {response}\n\n')
  prompt = input('Digite sua pergunta, ou escreva "FIM" para finalizar: ')

Digite sua pergunta, ou escreva "FIM" para finalizar? O que compete a Ouvidoria Ambiental?

Resposta: A Ouvidoria Ambiental e Agropecuária promove a interlocução entre a Administração Pública e o usuário, recebendo e tratando reclamações, denúncias, elogios, solicitações e sugestões relativas ao meio ambiente, saneamento básico, agricultura e agropecuária.


Esperando prompt: Quero reclamar sobre uma semente podre que recebi da EMATER

Resposta: Entendo sua frustração com a semente recebida da EMATER. Para te ajudar da melhor maneira possível, você poderia me dizer qual a sua intenção com essa reclamação? Você gostaria de receber uma nova semente, denunciar a qualidade das sementes, ou apenas registrar sua insatisfação com o ocorrido? 

Com essa informação, posso te direcionar para a Ouvidoria mais adequada para tratar sua demanda. 😉 



Esperando prompt: Quero receber uma nova semente

Resposta: Olá! Para solicitar uma nova semente, recomendo que você entre em contato diretamente com 